In [6]:
import os
import pretty_midi
import numpy as np
import test
import librosa
import math

In [7]:
PATH_TO_AUDIO = "../CSD/english/wav/"
sampling_rate = 16000
block_size = 128

In [12]:
tpb = block_size * 1.0 / sampling_rate
cb = math.ceil(.05 / tpb)
print(tpb, cb)

0.008 7


In [19]:
PHONEMES = ['sil', 'b','d','f','g','h','j','k','l','m','n','p','r','s','t','v','w','z','zh','ch','sh','th','dh','ng','y','ae','ei','e','ii','i','ai','a','ou','u','ao','uu','oi','au','eo','er','oo']
PHONEME2ID={}
for i,p in enumerate(PHONEMES):
    PHONEME2ID[p] = i

In [35]:
def parse_single_file(key, wav_path):
    midi_path = wav_path.replace('wav', 'mid')
    phoneme_path = wav_path.replace('wav', 'txt')
    
    y,sr = librosa.load(wav_path)
    y_resampled = librosa.resample(y,orig_sr=sr,target_sr=sampling_rate)

    midi = test.load_midi(midi_path)
    starts = [m.start for m in midi]
    ends = [m.end for m in midi]
    pitches = [int(m.pitch) for m in midi]
    syllables = test.load_text(phoneme_path)

    assert(len(midi) == len(syllables))
    
    label_length = math.ceil(y.shape[0] * 1.0 / block_size)
    phoneme_labels = np.zeros(label_length)
    for i in range(len(midi)):
        start_frame = math.floor(midi[i].start * float(sampling_rate) / float(block_size))
        end_frame = math.ceil(midi[i].end * float(sampling_rate) / float(block_size))
        
        phonemes = syllables[i].split("_")
        N = len(phonemes)
        duration = (end_frame - start_frame)
        durations = np.zeros(N, dtype='int32')
        for i, p in enumerate(phonemes):
            if(PHONEME2ID[p] < PHONEME2ID['ae']):
                durations[i] = cb
        total_const_time = np.sum(durations)
        n_const = total_const_time / cb
        n_vowel = N - n_const
        N_per_vowel = (duration - total_const_time) / n_vowel
        for i, p in enumerate(phonemes):
            if(PHONEME2ID[p] >= PHONEME2ID['ae']):
                durations[i] = N_per_vowel
        idx = start_frame
        for i, p in enumerate(phonemes):
            phoneme_labels[idx:(idx + durations[i])] = PHONEME2ID[p]
            idx += durations[i]
        
    return phoneme_labels
    

In [36]:
phoneme_label_dict = {}
for root, dirs, files in os.walk(PATH_TO_AUDIO):
    for name in files:
        if name.endswith('wav'):
            key = name.replace('.wav','')
            print(key)
            wav_path = os.path.join(root, name)
            phonemes = parse_single_file(key, wav_path)
            phoneme_label_dict[key] = phonemes

en001a
en001b
en002a
en002b
en003a
en003b
en004a
en004b
en005a
en005b
en006a
en006b
en007a
en007b
en008a
en008b
en009a
en009b
en010a
en010b
en011a
en011b
en012a
en012b
en013a
en013b
en014a
en014b
en015a
en015b
en016a
en016b
en017a
en017b
en018a
en018b
en019a
en019b
en020a
en020b
en021a
en021b
en022a
en022b
en023a
en023b
en024a
en024b
en025a
en025b
en026a
en026b
en027a
en027b
en028a
en028b
en029a
en029b
en030a
en030b
en031a
en031b
en032a
en032b
en033a
en033b
en034a
en034b
en035a
en035b
en036a
en036b
en037a
en037b
en038a
en038b
en039a
en039b
en040a
en040b
en041a
en041b
en042a
en042b
en043a
en043b
en044a
en044b
en045a
en045b
en046a
en046b
en047a
en047b
en048a
en048b
en049a
en049b
en050a
en050b


In [17]:
PHONEME2ID['ae']

25